![header](assets/header.png)

# Link Prediction - Introduction
In this Notebook we are going show to how to develop and train user-provided custom models with Amazon Neptune ML to perform node regression in a property graph.

<div style="background-color:#eeeeee; padding:10px; text-align:left; border-radius:10px; margin-top:10px; margin-bottom:10px; "><b>Note</b>: Make sure you're running the notebook with the <b>conda_neptune_ml_p36</b> jupyter kernel. This notebook take approximately 1 hour to complete</div>

[Neptune ML](https://docs.aws.amazon.com/neptune/latest/userguide/machine-learning.html#machine-learning-overview) is a feature of Amazon Neptune that enables users to automate the creation, management, and usage of Graph Neural Network (GNN) machine learning models within Amazon Neptune.  Neptune ML is built using [Amazon SageMaker](https://aws.amazon.com/sagemaker/) and [Deep Graph Library](https://www.dgl.ai/) and provides a simple and easy to use mechanism to build/train/maintain these models and then use the predictive capabilities of these models within a Gremlin query to predict elements or property values in the graph. 

This notebook follows the structure of the default [Neptune ML Link Prediction tutorial](https://github.com/aws/graph-notebook/blob/main/src/graph_notebook/notebooks/04-Machine-Learning/Neptune-ML-03-Introduction-to-Link-Prediction-Gremlin.ipynb) to make it easy to follow along and highligh the key differences when using a custom model with Neptune ML.

Link prediction is a unsupervised machine learning task where a model built using nodes and edges in the graph to predict whether an edge exists between two particular nodes. This task can be handled by Neptune ML's built in model, however, if you need to add customization to the model training step or want more granular control over the model used then you can implement your own custom model for the task.

Link prediction is commonly used to solve many common business problems such as:

* Predicting group membership in a social or identity network
* [Entity Resolution in an identity graph](https://github.com/awslabs/sagemaker-graph-entity-resolution/blob/master/source/sagemaker/dgl-entity-resolution.ipynb)
* Knowledge graph completion
* Product recommendation

Neptune ML uses a four step process to automate the process of creating production ready GNN models:

1. **Load Data** - Data is loaded into a Neptune cluster using any of the normal methods such as the Gremlin drivers or using the Neptune Bulk Loader.
2. **Export Data** - A service call is made specifying the machine learning model type and model configuration parameters.  The data and model configuration parameters are then exported from a Neptune cluster to an S3 bucket.
3. **Model Training** - A set of service calls are made to pre-process the exported data, train the machine learning model, and then generate an Amazon SageMaker endpoint that exposes the model.
4. **Run Queries** - The final step is to use this inference endpoint within our Gremlin queries to infer data using the machine learning model.

The **Model Training** part of the workflow when using Neptune ML's built-in models consist of:

* Data Processing API call
* Model Training API call
* Inference Endpoint API call

During the experimentation and development phase, when using custom models, there's an extra step between Data Processing API call and the Model Training API call for model implementation and local model testing.
So the overall **Model Training** part consists of:
* Data Processing API call
* ***Model development and local testing***
* Model Training API call
* Inference Endpoint API call


We'll use the [MovieLens 100k dataset](https://grouplens.org/datasets/movielens/100k/) provided by [GroupLens Research](https://grouplens.org/datasets/movielens/). This dataset consists of movies, users, and ratings of those movies by users. 


For this notebook we'll walk through how Neptune ML can predict product recommendations in a product knowledge graph.  To demonstrate this we'll predict the movies a user would be most likely to rate as well as which users are most likely to rate a given movie.  We'll walk through each step of loading and exporting the data, processing the data, developing and locally testing the model, training the model, and finally we'll show how to use that model to infer the movies a user has rated using Gremlin traversals. 

## Checking that we are ready to run Neptune ML 
Run the code below to check that this cluster is configured to run Neptune ML.

In [ ]:
import neptune_ml_utils as neptune_ml
neptune_ml.check_ml_enabled()

If the check above did not say that this cluster is ready to run Neptune ML jobs then please check that the cluster meets all the pre-requisites defined [here](https://docs.aws.amazon.com/neptune/latest/userguide/machine-learning.html#machine-learning-overview).

# Load the data
The first step in building a Neptune ML model is to load data into the Neptune cluster. Loading data for Neptune ML follows the standard process of ingesting data into Amazon Neptune, for this example we'll be using the Bulk Loader. 

We have written a script that automates the process of downloading the data from the MovieLens websites and formatting it to load into Neptune. All you need to provide is an S3 bucket URI that is located in the same region as the cluster.

<div style="background-color:#eeeeee; padding:10px; text-align:left; border-radius:10px; margin-top:10px; margin-bottom:10px; "><b>Note</b>: This is the only step that requires any specific input from the user, all remaining cells will automatically propogate the required values.</div>

In [ ]:
s3_bucket_uri="s3://<INSERT S3 BUCKET OR PATH>"
# remove trailing slashes
s3_bucket_uri = s3_bucket_uri[:-1] if s3_bucket_uri.endswith('/') else s3_bucket_uri

Now that you have provided an S3 bucket, run the cell below which will download and format the MovieLens data into a format compatible with Neptune's bulk loader.

In [ ]:
response = neptune_ml.prepare_movielens_data(s3_bucket_uri)

This process only takes a few minutes and once it has completed you can load the data using the `%load` command in the cell below.

In [ ]:
%load -s {response} -f csv -p OVERSUBSCRIBE --run

## Check to make sure the data is loaded

Once the cell has completed, the data has been loaded into the cluster. We verify the data loaded correctly by running the traversals below to see the count of nodes by label:  

<div style="background-color:#eeeeee; padding:10px; text-align:left; border-radius:10px; margin-top:10px; margin-bottom:10px; "><b>Note</b>: The numbers below assume no other data is in the cluster</div>

In [ ]:
%%gremlin
g.V().groupCount().by(label).unfold()

If our nodes loaded correctly then the output is:

* 19 genres
* 1682 movies
* 100000 rating
* 943 users

To check that our edges loaded correctly we check the edge counts:

In [ ]:
%%gremlin
g.E().groupCount().by(label).unfold()

If our edges loaded correctly then the output is:

* 100000 about
* 2893 included_in
* 100000 rated
* 100000 wrote


## Preparing for export

With our data validated let's remove a few `rated` vertices so that we can build a model that predicts these missing connections.  In a normal scenario, the data you would like to predict is most likely missing from the data being loaded so removing these values prior to building our machine learning model simulates that situation.

Specifically, let's remove the `rated` edgesfor `user_1`, to provide us with a few candidate vertices to run our link prediction tasks.  Let's start by taking a look at what `rated` edges currently exist.

In [ ]:
%%gremlin

g.V('user_1').outE('rated')

Now let's remove these edges to simulate them missing from our data.

In [ ]:
%%gremlin
g.V('user_1').outE('rated').drop()

Checking our data again we see that the edges have now been removed.

In [ ]:
%%gremlin
g.V('user_1').outE('rated')

# Export the data and model configuration

**Note:** Before exporting data ensure that Neptune Export has been configured as described here: [Neptune Export Service](https://docs.aws.amazon.com/neptune/latest/userguide/machine-learning-data-export-service.html#machine-learning-data-export-service-run-export)

With our product knowledge graph loaded we are ready to export the data and configuration which will be used to train the ML model.  

The export process is triggered by calling to the [Neptune Export service endpoint](https://docs.aws.amazon.com/neptune/latest/userguide/machine-learning-data-export-service.html).  This call contains a configuration object which specifies the type of machine learning model to build, in this example node classification, as well as any feature configurations required.  

<div style="background-color:#eeeeee; padding:10px; text-align:left; border-radius:10px; margin-top:10px; margin-bottom:10px; "><b>Note</b>: The configuration used in this notebook specifies only a minimal set of configuration options meaning that our model's predictions are not as accurate as they could be.  The parameters included in this configuration are one of a couple of sets of options available to the end user to tune the model and optimize the accuracy of the resulting predictions.</div>

The configuration options provided to the export service are broken into two main sections, selecting the target and configuring features. 

## Selecting the target

In the first section, selecting the target, we specify what type of machine learning task will be run. To run a link prediction mdoel do not specify any `targets` in the `additionalParams` value.  Unlike node classification or node regression, link prediction can be used to predict any edge type that exists in the graph between any two vertices.  Becasue of this, there is no need to define a target set of values.


## Configuring features
The second section of the configuration, configuring features, is where we specify details about the types of data stored in our graph and how the machine learning model should interpret that data.  In machine learning, each property is known as a feature and these features are used by the model to make predictions.  

In our export example below we have specified that the `title` property of our `movie` should be exported and trained as a `text_word2vec` feature, the `name` property of our `genre` should be exported and treated as a `category` feature, and that our `age` field should range from 0-100 and that data should be bucketed into 10 distinct groups. See the [documentation for feature encoding](https://docs.aws.amazon.com/neptune/latest/userguide/machine-learning-feature-encoding.html) to learn more about supported feature transformations in Neptune ML. 

<div style="background-color:#eeeeee; padding:10px; text-align:left; border-radius:10px; margin-top:10px; margin-bottom:10px; "><b>Important</b>: The example below is an example of a minimal amount of the features of the model configuration parameters and will not create the most accurate model possible.  Additional options are available for tuning this configuration to produce an optimal model are described here: <a href="https://docs.aws.amazon.com/neptune/latest/userguide/machine-learning-data-export-parameters.html">Neptune Export Process Parameters</a></div>

Running the cell below we set the export configuration and run the export process.  Neptune export is capable of automatically creating a clone of the cluster by setting `cloneCluster=True` which takes about 20 minutes to complete and will incur additional costs while the cloned cluster is running.  Exporting from the existing cluster takes about 5 minutes but requires that the `neptune_query_timeout` parameter in the [parameter group](https://docs.aws.amazon.com/neptune/latest/userguide/parameters.html) is set to a large enough value (>72000) to prevent timeout errors.

In [ ]:
export_params={ 
"command": "export-pg", 
"params": { "endpoint": neptune_ml.get_host(),
            "profile": "neptune_ml",
            "cloneCluster": False
            }, 
"outputS3Path": f'{s3_bucket_uri}/neptune-export',
"additionalParams": {
        "neptune_ml": {
          "version": "v2.0",
         "features": [
            {
                "node": "movie",
                "property": "title",
                "type": "word2vec"
            },
            {
                "node": "genre",
                "property": "name",
                "type": "category",
            },
            {
                "node": "user",
                "property": "age",
                "type": "bucket_numerical",
                "range" : [1, 100],
                "num_buckets": 10
            }
         ]
        }
      },
"jobSize": "medium"}

In [ ]:
%%neptune_ml export start --export-url {neptune_ml.get_export_service_host()} --export-iam --wait --store-to export_results
${export_params}

# ML data processing, model development, model training, and endpoint creation

Once the export job is completed we are now ready to train our machine learning model and create the inference endpoint. Training our Neptune ML model requires a few steps.  
  
<div style="background-color:#eeeeee; padding:10px; text-align:left; border-radius:10px; margin-top:10px; margin-bottom:10px; "><b>Note</b>: The cells below only configure a minimal set of parameters required to run a model training.</div>

## Data processing
The first step (data processing) processes the exported graph dataset using standard feature preprocessing techniques to prepare it for use by DGL. This step performs functions such as feature normalization for numeric data and encoding text features using word2vec. At the conclusion of this step the dataset is formatted for model training. 

This step is implemented using a SageMaker Processing Job and data artifacts are stored in a pre-specified S3 location once the job is complete.

Additional options and configuration parameters for the data processing job can be found using the links below:

* [Data Processing](https://docs.aws.amazon.com/neptune/latest/userguide/machine-learning-on-graphs-processing.html)
* [dataprocessing command](https://docs.aws.amazon.com/neptune/latest/userguide/machine-learning-api-dataprocessing.html)

Run the cells below to create the data processing configuration and to begin the processing job.

In [ ]:
# The training_job_name can be set to a unique value below, otherwise one will be auto generated
training_job_name=neptune_ml.get_training_job_name('link-prediction')

processing_params = f"""
--config-file-name training-data-configuration.json
--job-id {training_job_name} 
--s3-input-uri {export_results['outputS3Uri']} 
--s3-processed-uri {str(s3_bucket_uri)}/preloading """

In [ ]:
%neptune_ml dataprocessing start --wait --store-to processing_results {processing_params}

## Model development and local testing
The next step consists of developing the model, writing the model training code, and testing the custom implementation before full-fledged model training.

The custom model implementation is provided in the [`link-predict/src/`](link-predict/src/) directory which has the following contents.

* **[`model-hpo-configuration.json`](link-predict/src/model-hpo-configuration.json)** contains the configuration for the model hyperparameters, their defaults and ranges according to the [Neptune ML hyperparameter configuration specificaton](https://docs.aws.amazon.com/neptune/latest/userguide/machine-learning-customizing-hyperparams.html)
* **[`train.py`](link-predict/src/train.py)** is the entrypoint script for model training. It contains the custom model code implementation, and when executed, runs model training with the hyperparameter values passed in as program arguments.
* **[`transform.py`](link-predict/src/transform.py)** is the entrypoint script for [model transform](https://docs.aws.amazon.com/neptune/latest/userguide/machine-learning-model-transform.html). When executed, it computes the model artifacts that will be needed when the model is deployed to an inference endpoint


Let's take a look at the training script in more detail

In [ ]:
!pygmentize link-predict/src/train.py

As you can see the `train.py` defines the `RGCNLinkPrediction` model using the implementation provided by the Neptune ML Toolkit modelzoo. The modelzoo contains reusable modular GNN components implemented in [DGL](https://github.com/dmlc/dgl) with the Pytorch backend, which you can use in your custom implementations.

The script also implements functions to train this model architecture using pytorch.

We can run this script locally, to test it out and really see the training in action but first we need to bring the processed data locally and create a folder to store the training output.

In [ ]:
s3_processed_data = f"""{str(s3_bucket_uri)}/preloading"""

!aws s3 cp --recursive $s3_processed_data link-predict/data/

!mkdir output/

Now let's run the training script

In [ ]:
!cd link-predict && python src/train.py --local

This should have run succesfully for a few epochs and generated some files in `output/`.
The model parameters are in `output/model.pt` and evaluation metrics for the training run are in `output/eval_metrics_info.json`.
The contents of output are what will be generated when we train with the Neptune ML modeltraining API later.


Let's also take a look at the transform script in detail

In [ ]:
!pygmentize src/transform.py

The transform script is what is used to compute the model artifacts such as node embeddings and precomputed predictions needed for inference.

The transform script can also be run on the local instance.

In [ ]:
!cd node-class && python src/transform.py --local

The transform script should have added even more files like `entity.npy` to the `output/` folder.
These files are needed to create the model that will be deployed but are not generated by `train.py`.
This is because when we train with the Neptune ML modeltraining API, `train.py` is executed by multiple SageMaker training jobs trying different hyperparameter configurations.
However, only the artifacts from the best training job are needed for the deployment.
Therefore, when using the Neptune ML train API, `transform.py` is called once on the output from the best training instance.


Now, that we have verified locally that both of the entry point scripts work, we can move to the next of training using the Neptune ML modeltraining API.

To do this we must first upload the `link-predict/src/` to S3 so that the neptune ml modeltraining API can pick our custom model implementation.

In [ ]:
s3_custom_source_location = f"""{str(s3_bucket_uri)}/training/source/{training_job_name}"""

!aws s3 cp --recursive link-predict/src/ $s3_custom_source_location

## Model training
The next step (model training) trains the ML model that will be used for predictions. The model training is done in stages. The first stage uses a SageMaker Processing job to parse the `model-hpo-configuration.json` and generate a model training strategy. A model training strategy is a configuration set that specifies what type of model and model hyperparameter ranges will be used for the model training. Once the first stage is complete, the SageMaker Processing job launches a SageMaker Hyperparameter tuning job. The SageMaker Hyperparameter tuning job runs a pre-specified number of model training job trials (`train.py`) on the processed data. Once all the training jobs are complete, the Hyperparameter tuning job also notes the training job that produced the best performing model.
The SageMaker Processing job then computes and stores (`transform.py`) the model artifacts generated by the best SageMaker training job in the output S3 location.

Additional options and configuration parameters for the data processing job can be found using the links below:

* [Model Training](https://docs.aws.amazon.com/neptune/latest/userguide/machine-learning-on-graphs-model-training.html)
* [modeltraining command](https://docs.aws.amazon.com/neptune/latest/userguide/machine-learning-api-modeltraining.html)

<div style="background-color:#eeeeee; padding:10px; text-align:left; border-radius:10px; margin-top:10px; margin-bottom:10px; "><b>Information</b>: Link prediction is a more computationally complex model than classification or regression so training this model will take 2-3 hours</div>

In [ ]:
training_params=f"""
--job-id {training_job_name} 
--data-processing-id {training_job_name} 
--instance-type ml.p3.2xlarge
--s3-output-uri {str(s3_bucket_uri)}/training """

In [ ]:
# %neptune_ml training start --wait --store-to training_results {training_params}

curl \
  -X POST https://(your Neptune endpoint)/ml/modeltraining
  -H 'Content-Type: application/json' \
  -d '{
        "id" : "<INSERT training job name from above>",
        "dataProcessingJobId" : "<INSERT training job name from above>",
        "trainModelS3Location" : "<INSERT S3 BUCKET OR PATH>",
        "trainingInstanceType" : "ml.g4.xlarge"
        "customModelParameters" : {
            "sourceS3DirectoryPath": "<INSERT Source S3 path from above>",
            "trainingEntryPointScript": "train.py",
            "transformEntryPointScript": "transform.py"
            }
      }'

Once the training job is completed, you will have a model that was trained using the custom implementation that was provided.

After this stage, all the following steps for endpoint creation and inference are identical to using the built-in Neptune ML models.

## Endpoint creation
The final step is to create the inference endpoint which is an Amazon SageMaker endpoint instance that is launched with the model artifacts produced by the best training job. This endpoint will be used by our graph queries to  return the model predictions for the inputs in the request. The endpoint once created stays active until it is manually deleted. Each model is tied to a single endpoint.

Additional options and configuration parameters for the data processing job can be found using the links below:

* [Inference Endpoint](https://docs.aws.amazon.com/neptune/latest/userguide/machine-learning-on-graphs-inference-endpoint.html)
* [Endpoint command](https://docs.aws.amazon.com/neptune/latest/userguide/machine-learning-api-endpoints.html)

<div style="background-color:#eeeeee; padding:10px; text-align:left; border-radius:10px; margin-top:10px; margin-bottom:10px; "><b>Information</b>: The endpoint creation process takes ~5-10 minutes</div>

In [ ]:
endpoint_params=f"""
--job-id {training_job_name} 
--model-job-id {training_job_name}"""

In [ ]:
%neptune_ml endpoint create --wait --store-to endpoint_results {endpoint_params}

Once this has completed we get the endpoint name for our newly created inference endpoint.  The cell below will set the endpoint name which will be used in the Gremlin queries below.   

In [ ]:
endpoint=endpoint_results['endpoint']['name']

# Querying using Gremlin

Now that we have our inference endpoint setup let's query our product knowledge graph to show how to predict how likely it is that a user will rate a movie.  The need to predict the likelyhood of connections in a product knowledge graph is commonly used to provide recommendations for products that a customer might purchase.

Unlike node classification and node regression, link prediction can infer any of the edge labels that existed in our graph when the model was created.  In our model this means we could infer the probability that a `wrote`, `about`, `rated`, or `included_in` edge exists between any two vertices.  However for this example we are going to focus on inferring the `rated` edges between the `user` and `movie` vertices. 

## Predicting what movies a user will rate

Before we predict what movies `user_1` is most likely to rate let's verify that our graph does not contain any `rated` edges for `user_1`.


In [ ]:
%%gremlin
g.V('user_1').out('rated').hasLabel('movie').valueMap()

As expected, their are not any `rated` edges for `user_1`.  Maybe `user_1` is a new user in our system and we want to provide them some product recommendations.  Let's modify the query to predict what movies `user_1` is most likely to rate. 

First, we add the `with()` step to specify the inference endpoint we want to use with our Gremlin query like this
`g.with("Neptune#ml.endpoint","<INSERT ENDPOINT NAME>")`.  

<div style="background-color:#eeeeee; padding:10px; text-align:left; border-radius:10px; margin-top:10px; margin-bottom:10px; "><b>Note</b>: The endpoint values are automatically passed into the queries below</div>

Second, when we ask for the link within our query we use the `out()` step to predict the target node or the `in()` step to predict the source node.  For each of these steps we need to specify the type of model being used with a with() step (`with("Neptune#ml.prediction")`).

Putting these items together we get the query below, which returns the movies that` user_1` is likely to rate.

In [ ]:
%%gremlin
g.with("Neptune#ml.endpoint","${endpoint}").
    V('user_1').out('rated').with("Neptune#ml.prediction").hasLabel('movie').valueMap('title')

Great, we can now see that we are predicted edges showing that `Sleepers` is the movie that `user_1` is most likely to rate.  In the example above we predicted the target node but we can also use the same mechanism to predict the source node.  

Let's turn that question around and say that we had a product and we wanted to find the people most likely to rate this product.

## Predicting the top 10 users most likely to rate a movie
To accomplish this we would want to start at the movie vertex and predict the rated edge back to the user.  Since we want to return the top 10 recommended users we need to use the `.with("Neptune#ml.limit",10)` configuration option.  Combining these together we get the query below which finds the top 10 users most likely to rate `Apollo 13`.

In [ ]:
%%gremlin
g.with("Neptune#ml.endpoint","${endpoint}").
    with("Neptune#ml.limit",10).
      V().has('title', 'Apollo 13 (1995)').
        in('rated').with("Neptune#ml.prediction").hasLabel('user').id()

With that we have sucessfully been able to show how you can use link prediction to predict edges starting at either end.

From the examples we have shown here you can begin to see how the ability to infer unknown connections within a graph starts to enable many interesting and unique use cases within Amazon Neptune.  

# Cleaning Up 
Now that you have completed this walkthrough you have created a Sagemaker endpoint which is currently running and will incur the standard charges.  If you are done trying out Neptune ML and would like to avoid these recurring costs, run the cell below to delete the inference endpoint.

In [ ]:
neptune_ml.delete_endpoint(training_job_name)

In addition to the inference endpoint the CloudFormation script that you used has setup several additional resources.  If you are finished then we suggest you delete the CloudFormation stack to avoid any recurring charges. For instructions, see Deleting a Stack on the AWS CloudFormation Console. Be sure to delete the root stack (the stack you created earlier). Deleting the root stack deletes any nested stacks.